In [3]:
InteractiveShell.ast_node_interactivity = "all"

# 1. Deserialization

### 2020-09-10 11:40:05.000 to 2020-09-17 11:40:07.000 

In [47]:
sensor1 = requests.get("https://api.app.geoctrl.com/v1/data/b8-27-eb-5a-a7-e1?limit=50000&page=0&start=2020-09-10T11%3A40%3A06.000Z&end=2020-09-17T11%3A40%3A06.000Z",
         auth=HTTPBasicAuth("3085089319","03bb54cc-f13b-4a5c-b070-9d3ddf31615a"))
sensor2 = requests.get("https://api.app.geoctrl.com/v1/data/b8-27-eb-9d-4d-12?limit=50000&page=0&start=2020-09-10T11%3A40%3A06.000Z&end=2020-09-17T11%3A40%3A06.000Z",
         auth=HTTPBasicAuth("3085089319","03bb54cc-f13b-4a5c-b070-9d3ddf31615a"))

In [48]:
sensor1_text = sensor1.text
sensor2_text = sensor2.text

In [49]:
data1 = json.loads(sensor1_text)
print(type(data1))
data2 = json.loads(sensor2_text)
print(type(data2))

<class 'dict'>
<class 'dict'>


In [50]:
#use get method: to see the key you want to look at. 
#keys method, values method, iterate through it via for loop or use .items to 
#iterate through key, value pairs. 


# 2. Manipulate Dictionary to something usable for analysis

### 2.1. first sensor1

In [51]:
print(data1.keys())
# print(data1.values()) # we dont have any use for "id", "data", "links"
# as the keys in the dictionary, we need to move them to 

dict_keys(['id', 'data', 'links'])


In [52]:
# try and separate out the data into pandas dataframe perhaps
data_access1 = data1["data"]

In [53]:
type(data_access1)

list

In [ ]:
data_access1 # hmm, list of dictionaries

In [55]:
# # to access list use integers not strings
# for data_primary in data_access1:
#     print(data_primary)

In [56]:
# type(data_primary)

In [41]:
# print(data_primary.keys())
# its only giving last value not entire thing
# print(data_primary.values())

In [36]:
# def getall_data1():
#     data_access1 = data1["data"]
#     for data_primary in data_access1:
#         print(data_primary)

In [57]:
# list of dictionaries can be directly converted to dataframe
# no need to go through steps above.
sensor1data = pd.DataFrame(data_access1)

In [58]:
sensor1data.head()
sensor1data.shape

,src,rand,hash,rssi,ts
0,bt,False,00:25:00:51:ae:39,-83,2020-09-10T11:40:06.000Z
1,bt,False,e4:85:01:fb:58:73,-85,2020-09-10T11:40:11.000Z
2,bt,False,00:25:00:51:ae:39,-89,2020-09-10T11:40:11.000Z
3,wifi,False,08:17:35:8c:2d:45,-48,2020-09-10T11:40:14.000Z
4,wifi,False,1c:bf:ce:22:7a:8c,-22,2020-09-10T11:40:14.000Z


(50000, 5)

### 2.2. repeat for sensor 2

In [59]:
data_access2 = data2["data"]

In [ ]:
data_access2

In [61]:
sensor2data = pd.DataFrame(data_access2)

In [ ]:
sensor2data.head(50)
sensor2data.shape

In [63]:
# 5afe4e86b71b4271a7094102201510 : api key for world weatheronline
# hash probably refers to geohashing, pretty much location indicators.
# “dwell time” = amount of time it is reasonable to  assume a unique id has been continuously “visible”.
# think the answer is in the differentials of time stamps regardless of connection strength. 
sensor1data.to_excel("sensor1.xlsx", sheet_name="sensor1", index=False)
sensor2data.to_excel("sensor2.xlsx", sheet_name="sensor2", index=False)

# 3. Analysis Chunk

### 3.1. First sensor PK92

In [65]:
sensor1data.head()

,src,rand,hash,rssi,ts
0,bt,False,00:25:00:51:ae:39,-83,2020-09-10T11:40:06.000Z
1,bt,False,e4:85:01:fb:58:73,-85,2020-09-10T11:40:11.000Z
2,bt,False,00:25:00:51:ae:39,-89,2020-09-10T11:40:11.000Z
3,wifi,False,08:17:35:8c:2d:45,-48,2020-09-10T11:40:14.000Z
4,wifi,False,1c:bf:ce:22:7a:8c,-22,2020-09-10T11:40:14.000Z


In [68]:
# use regex to separate out the timestamp, the days are the same for both the sensors. 
# regex code : (?:[01]\d|2[0-3]):(?:[0-5]\d):(?:[0-5]\d)

data = []
for line in sensor1data["ts"]:
    time = re.search("(?:[01]\d|2[0-3]):(?:[0-5]\d):(?:[0-5]\d)", str(line))
    if time:
        data.append(time.group(0))
    else:
        data.append("no match")
sensor1data["timestamp"] = data
sensor1data.head()

,src,rand,hash,rssi,ts,timestamp
0,bt,False,00:25:00:51:ae:39,-83,2020-09-10T11:40:06.000Z,11:40:06
1,bt,False,e4:85:01:fb:58:73,-85,2020-09-10T11:40:11.000Z,11:40:11
2,bt,False,00:25:00:51:ae:39,-89,2020-09-10T11:40:11.000Z,11:40:11
3,wifi,False,08:17:35:8c:2d:45,-48,2020-09-10T11:40:14.000Z,11:40:14
4,wifi,False,1c:bf:ce:22:7a:8c,-22,2020-09-10T11:40:14.000Z,11:40:14


In [77]:
# capture time differential
sensor1data.dtypes

src                  object
rand                   bool
hash                 object
rssi                  int64
ts                   object
timestamp    datetime64[ns]
dtype: object

In [109]:
# just making sure its in right format
sensor1data["timestamp"] = pd.to_datetime(sensor1data['timestamp'])

In [110]:
sensor1data["deltaT"] = sensor1data["timestamp"].diff().dt.seconds.div(60, fill_value = 0)

In [111]:
sensor1data["deltaT"] = sensor1data["deltaT"].round(2)

In [112]:
sensor1data.head()

,src,rand,hash,rssi,ts,timestamp,deltaT
0,bt,False,00:25:00:51:ae:39,-83,2020-09-10T11:40:06.000Z,2020-10-15 11:40:06,0.00
1,bt,False,e4:85:01:fb:58:73,-85,2020-09-10T11:40:11.000Z,2020-10-15 11:40:11,0.08
2,bt,False,00:25:00:51:ae:39,-89,2020-09-10T11:40:11.000Z,2020-10-15 11:40:11,0.00
3,wifi,False,08:17:35:8c:2d:45,-48,2020-09-10T11:40:14.000Z,2020-10-15 11:40:14,0.05
4,wifi,False,1c:bf:ce:22:7a:8c,-22,2020-09-10T11:40:14.000Z,2020-10-15 11:40:14,0.00


In [122]:
sensor1data.loc[sensor1data['deltaT'] >= 1] #1 break taken at 13:13

,src,rand,hash,rssi,ts,timestamp,deltaT
6895,bt,False,00:25:00:51:ae:39,-83,2020-09-10T13:15:50.000Z,2020-10-15 13:15:50,2.37


### 3.2. Second Sensor PK106

In [98]:
sensor2data.head()

,src,rand,hash,rssi,ts
0,bt,False,64:a2:f9:2d:1d:d7,-43,2020-09-14T09:16:39.000Z
1,bt,False,64:a2:f9:2d:1d:d7,-46,2020-09-14T09:16:44.000Z
2,bt,False,e4:85:01:45:64:f4,-81,2020-09-14T09:16:44.000Z
3,bt,False,64:a2:f9:2d:1d:d7,-40,2020-09-14T09:16:49.000Z
4,bt,False,64:a2:f9:2d:1d:d7,-39,2020-09-14T09:16:54.000Z


In [99]:
data = []
for line in sensor2data["ts"]:
    time = re.search("(?:[01]\d|2[0-3]):(?:[0-5]\d):(?:[0-5]\d)", str(line))
    if time:
        data.append(time.group(0))
    else:
        data.append("no match")
sensor2data["timestamp"] = data
sensor2data.head()

,src,rand,hash,rssi,ts,timestamp
0,bt,False,64:a2:f9:2d:1d:d7,-43,2020-09-14T09:16:39.000Z,09:16:39
1,bt,False,64:a2:f9:2d:1d:d7,-46,2020-09-14T09:16:44.000Z,09:16:44
2,bt,False,e4:85:01:45:64:f4,-81,2020-09-14T09:16:44.000Z,09:16:44
3,bt,False,64:a2:f9:2d:1d:d7,-40,2020-09-14T09:16:49.000Z,09:16:49
4,bt,False,64:a2:f9:2d:1d:d7,-39,2020-09-14T09:16:54.000Z,09:16:54


In [114]:
#make sure its in the right format
sensor2data["timestamp"] = pd.to_datetime(sensor2data["timestamp"])

In [ ]:
sensor2data.dtypes

In [116]:
sensor2data["deltaT"] = sensor2data["timestamp"].diff().dt.seconds.div(60, fill_value = 0)


In [117]:
sensor2data["deltaT"] = sensor2data["deltaT"].round(2)

In [118]:
sensor2data.head()

,src,rand,hash,rssi,ts,timestamp,deltaT
0,bt,False,64:a2:f9:2d:1d:d7,-43,2020-09-14T09:16:39.000Z,2020-10-15 09:16:39,0.00
1,bt,False,64:a2:f9:2d:1d:d7,-46,2020-09-14T09:16:44.000Z,2020-10-15 09:16:44,0.08
2,bt,False,e4:85:01:45:64:f4,-81,2020-09-14T09:16:44.000Z,2020-10-15 09:16:44,0.00
3,bt,False,64:a2:f9:2d:1d:d7,-40,2020-09-14T09:16:49.000Z,2020-10-15 09:16:49,0.08
4,bt,False,64:a2:f9:2d:1d:d7,-39,2020-09-14T09:16:54.000Z,2020-10-15 09:16:54,0.08


In [123]:
sensor2data.loc[sensor2data['deltaT'] >= 1] # from 9:21 to 15:45 break was taken

,src,rand,hash,rssi,ts,timestamp,deltaT
545,bt,False,78:ab:bb:27:41:07,-82,2020-09-14T15:45:13.000Z,2020-10-15 15:45:13,383.27


# 4. Get some weather data

In [125]:
weather = requests.get("https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=5afe4e86b71b4271a7094102201510&q=Zurich&format=json&date=2020-09-10&enddate=2020-09-15")

In [128]:
weather_text = weather.text

In [176]:
weatherz = json.loads(weather_text)
print(type(weatherz))

<class 'dict'>


In [ ]:
weatherz

In [178]:
w1 = weatherz["data"]

In [179]:
type(w1) #still not a list

dict

In [180]:
w2 = w1["weather"] # question access

In [ ]:
w2

In [182]:
# extracting multiple elements from a list with indices 0,4 
# these are the dates I am interested in. 
w3 = [w2[i] for i in (0,4)] #1

In [ ]:
w3 # this has list items that has dictionaries. 

In [ ]:
for weather_data in w3: #2
    print(weather_data)

In [197]:
type(weather_data)

dict

In [198]:
weather_data.keys()

dict_keys(['date', 'astronomy', 'maxtempC', 'maxtempF', 'mintempC', 'mintempF', 'avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex', 'hourly'])

In [199]:
hourly_access = weather_data["hourly"] #3

In [ ]:
hourly_access #this is a list

In [ ]:
for time_data in hourly_access:   #4 #a dictionary
    print(time_data)

In [202]:
cloud_cover = time_data["cloudcover"] #5

In [203]:
cloud_cover

'4'

In [232]:
#combine them all together 

def get_cloud_cover():
    w3 = [w2[i] for i in (0,4)]
    for weather_data in w3:
        hourly_access = weather_data["hourly"]
        for time_data in hourly_access:
            cloud_cover = time_data["cloudcover"]
            cloudyness.append(cloud_cover)

In [233]:
get_cloud_cover()

In [231]:
cloudyness = []

In [ ]:
cloudyness

In [235]:
cloudyness  = DataFrame(cloudyness, columns=["cloudscale"])

In [236]:
hours = [0, 3, 6, 9, 12, 15, 18, 21, 0, 3, 6, 9, 12, 15, 18, 21]

In [237]:
cloudyness["hours"] = hours

In [238]:
list1 = ["PK92", "PK92", "PK92", "PK92", "PK92", "PK92", "PK92", "PK92", "PK106","PK106","PK106","PK106", "PK106", "PK106", "PK106", "PK106"]
list2 = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
cloudyness["id"] = list1
cloudyness["breakstaken"] = list2

In [239]:
cloudyness.head()

,cloudscale,hours,id,breakstaken
0,18,0,PK92,0
1,18,3,PK92,0
2,13,6,PK92,0
3,9,9,PK92,0
4,9,12,PK92,1


In [245]:
cloudyness.to_excel("cloud.xlsx", sheet_name = "cloud", index=False)